# Hadamard Test

## Overview

- Type: Versatile quantum gadget and subroutine
- Approachability: Easy
- Prerequisites: Phase kickback
- Postrequisites: Phase estimation

The Hadamard test is a simple circuit for calculating the expectation value of a unitary operator. It takes advantage of phase kickback to encode the expectation value in the amplitudes of a single qubit auxiliary register. 

## Description

### Pure state

Let $ \langle U \rangle = \langle \psi \vert U \vert \psi \rangle$ be the expectation value of $U$ with respect to a pure state $\vert \psi \rangle$. The algorithm proceeds by measuring the real and imaginary parts of $\langle U \rangle$ by inserting or removing a phase gate from the following circuit.

**Place circuit here, include input state rho, controlled unitary CU, and single auxiliary qubit. Dashed box representing the phase gate**

#### Real part

After applying the circuit without a phase gate, prior to measurement, the resulting state is 

$$
\vert 0 \rangle \otimes \frac{\vert\psi \rangle + U \vert \psi \rangle}{2} + \vert 1 \rangle \otimes \frac{\vert\psi\rangle - U \vert\psi\rangle}{2}.
$$

The probability $\Pr(0)$ of measuring 'zero' in the first register is given by the Born rule.

$$
\begin{aligned}
\Pr(0) &= \frac14 (2 + \langle U \rangle + \langle U^\dagger \rangle) \\
&= \frac{1 + \mathrm{Re}\langle U \rangle}{2}
\end{aligned}
$$

Here $\mathrm{Re}$ is the real part. Of course, $\Pr(1) = 1- \Pr(0)$. By measuring the circuit $N$ times, the probability $\Pr(0)$, hence $\mathrm{Re} \langle U \rangle$, can be measured to accuracy $O(1/\sqrt{N})$. However, this is not the optimal approach. By using amplitude amplification instead of repeated measurements, a quadratic improvement can be obtained, boosting the accuracy to an optimal $O(1/N)$ known as the 'Heisenberg limit.' 

#### Imaginary part

With the phase gate inserted, the final output prior to measurement acquires a factor of $i$ in front of the $U$.

$$
\vert 0 \rangle \otimes \frac{\vert\psi \rangle + i U \vert \psi \rangle}{2} + \vert 1 \rangle \otimes \frac{\vert\psi\rangle - i U \vert\psi\rangle}{2}.
$$

Now, the probability $\Pr(0)$ of measuring 'zero' in the first register is given by

$$
\begin{aligned}
\Pr(0) &= \frac14 (2 + i\langle U \rangle -i \langle U^\dagger \rangle) \\
&= \frac{1 - \mathrm{Im}\langle U \rangle}{2}
\end{aligned}
$$

Thus, the imaginary part can be obtained in an entirely analogous manner: either through repeated measurements or amplitude amplification.

## Applications

### Phase estimation

Suppose the prepared state $\vert \psi \rangle = \vert u \rangle$ is an eigenstate of $U$ with eigenvalue $e^{2\pi i \theta}$ for $\theta \in [0,1)$, so that $\langle U \rangle= e^{2\pi i \theta} = \cos(2\pi \theta) + i \sin(2\pi \theta)$. Prior to measurement, the Hadamard circuit without phase gate yields

$$
\begin{aligned}
\left(\frac{1 + e^{2\pi i \theta}}{2} \vert 0 \rangle + \frac{1 - e^{2\pi i \theta}}{2} \vert 1 \rangle\right) \otimes \vert u \rangle.
\end{aligned}
$$

The amplitudes encode information about the phase $\theta$, and any amplitude estimation technique can be used to learn $\theta$. Below we consider the simple but suboptimal estimate via direct measurements. The probabilities are given by

$$
\begin{aligned}
    \Pr(0) &= \frac{1+\mathrm{Re}\langle U \rangle}{2} = \frac{1 + \cos 2\pi \theta}{2} = \cos^2(\pi \theta) \\
    \Pr(1) &= \sin^2(\pi\theta).
\end{aligned}
$$

These probabilities determine $\theta$ up to two possible values. The phase-gated Hadamard circuit resolves the ambiguity.

$$
\begin{aligned}
    \Pr(0) &= \frac{1-\mathrm{Im}\langle U \rangle}{2} = \frac{1 - \sin 2\pi \theta}{2} = \sin^2(\pi \vert\theta- 1/4\vert) \\
    \Pr(1) &= \cos^2(\pi \vert\theta- 1/4\vert).
\end{aligned}
$$

### SWAP test for pure state fidelity



## Mixed State Hadamard Test

### Applications: Purity

## Generalization: Block encoding

When an operator $O$ is block-encoded in a unitary $U$ via $(\langle B \vert \otimes I) U (\vert B \rangle \otimes I) = O$, $\langle O \rangle = \langle \psi \vert O \vert \psi \rangle$ can be obtained provided $\vert B \rangle$ can be accurately and efficiently prepared.